# Filesystem Operations (os & shutil)

- DevOps scripts often need to create, delete, copy, and move files and directories as part of automation workflows.  
- The `os` module provides low-level filesystem functions, while `shutil` offers higher-level operations like copying and recursive removal.  
- These tools work hand-in-hand with `pathlib` (for path manipulation) to build robust file management scripts.
 
## Listing Directory Contents

- Use `os.listdir(path)` to get a list of entry names (files and subdirectories) in a directory.  
- Use `Path(path).iterdir()` to iterate over `Path` objects, which you can query further with methods like `.is_file()` or `.is_dir()`.  
- `os.listdir` returns a plain list of strings; `iterdir()` yields full `Path` objects, making downstream operations more convenient.  

In [7]:
import os 
from pathlib import Path 
import shutil

"""
Directory structure:

temp_listing_dir/
|-- file1.txt
|-- file2.log
|-- subdir1/
       |-- subfile.py
"""

tmp_path = Path('temp_listing_dir')
tmp_path.mkdir(exist_ok=True)
(tmp_path / 'file1.txt').touch()
(tmp_path / 'file2.log').touch()
(tmp_path /'subdir1').mkdir(exist_ok=True)
(tmp_path /'subdir1' /'subfile.py').touch()


print(f"--- os.listdir(\"{tmp_path}\") ---")
for name in os.listdir(tmp_path):
    print(type(name).__name__, name)


print(f"\n--- Path (\"{tmp_path}\").iterdir() ---")
for entry in tmp_path.iterdir():
    print(entry)

shutil.rmtree(tmp_path)

--- os.listdir("temp_listing_dir") ---
str file1.txt
str file2.log
str subdir1

--- Path ("temp_listing_dir").iterdir() ---
temp_listing_dir\file1.txt
temp_listing_dir\file2.log
temp_listing_dir\subdir1


## Creating Directories

- `os.mkdir(path)` creates a single directory and fails if parents don’t exist or if it already exists.  
- `os.makedirs(path, exist_ok=False)` creates all intermediate directories; set `exist_ok=True` to ignore existing leaf.  
- `Path(path).mkdir(parents=True, exist_ok=True)` is the pathlib equivalent for recursive, idempotent creation.  

In [13]:
from pathlib import Path
import shutil

single = Path("my_single_dir")
try:
    single.mkdir(exist_ok=True)
    print(f"Created {single}: {single.exists()}")
finally:
    if single.exists():
        single.rmdir()    


nested = Path("parent/child/grandchild")
nested.mkdir(parents=True, exist_ok=True)
print(f"Created nested path {nested}: {nested.exists()}")

shutil.rmtree("parent")

Created my_single_dir: True
Created nested path parent\child\grandchild: True


## Removing Files and Directories

- `os.remove(path)` or `Path(path).unlink()` deletes a single file and raises if missing (unless `missing_ok=True`).  
- `os.rmdir(path)` or `Path(path).rmdir()` removes an **empty** directory only.  
- `shutil.rmtree(path)` recursively deletes a directory tree and all contents; use with extreme caution.  

## Copying Files and Directories

- `shutil.copy(src, dst)` copies a file but does not preserve metadata like timestamps or permissions.  
- `shutil.copy2(src, dst)` copies files **and** attempts to preserve metadata.  
- `shutil.copytree(src_dir, dst_dir, dirs_exist_ok=False)` recursively copies an entire directory tree.  

## Moving Files and Directories

- Use `shutil.move(src, dst)` to move or rename files and directories in one step.  
- If `dst` is an existing directory, `src` is moved **into** it; if `dst` names a file, `src` is renamed there.  
- Moving across filesystems may involve a copy-and-delete under the hood.  

## Common Pitfalls & How to Avoid Them

- **PermissionError:** Operations fail if the script lacks rights. Ensure correct ownership or run with appropriate privileges.  
- **Non-empty Directories:** `os.rmdir()` and `Path.rmdir()` only remove empty dirs. Use `shutil.rmtree()` for recursive deletion, but do so carefully.  
- **Existing Destinations:** `shutil.copytree()` errors if the target exists unless `dirs_exist_ok=True`. Consider pre-cleanup or that flag.  
- **Irreversible Deletions:** There is no undo for `os.remove`, `os.rmdir`, or `shutil.rmtree()`. Add confirmation or dry-run modes when deleting!